# Exploratory data analysis

## 1. Dataset description




## 2. Load the dataset

In [1]:
library(testthat)
library(MASS)
library(tidyverse)

Warning message:
"package 'testthat' was built under R version 3.6.2"
-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::is_null() masks testthat::is_null()
x dplyr::lag()     masks stats::lag()
x dplyr::matches() masks tidyr::matches(), testthat::matches()
x dplyr::select()  masks MASS::select()



In [2]:
url <- "https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series"
covid19_confirmed <- read_csv(paste(url, "/time_series_covid19_confirmed_global.csv?raw=true", sep = ""))
covid19_death <- read_csv(paste(url, "/time_series_covid19_deaths_global.csv?raw=true", sep = ""))

Parsed with column specification:
cols(
  .default = col_double(),
  `Province/State` = col_character(),
  `Country/Region` = col_character()
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .default = col_double(),
  `Province/State` = col_character(),
  `Country/Region` = col_character()
)

See spec(...) for full column specifications.



In [3]:
head(covid19_confirmed)

Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,840,906,933,996,1026,1092,1176,1279,1351,1463
NA,Albania,41.1533,20.1683,0,0,0,0,0,0,...,518,539,548,562,584,609,634,663,678,712
NA,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2268,2418,2534,2629,2718,2811,2910,3007,3127,3256
NA,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,673,696,704,713,717,717,723,723,731,738
NA,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,19,19,24,24,24,24,25,25,25,25
NA,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,23,23,23,23,23,23,24,24,24,24


In [4]:
country_data <- read_csv("../data/clean_data/country_data.csv")

Parsed with column specification:
cols(
  country = col_character(),
  age_1564 = col_double(),
  age_64up = col_double(),
  age_0014 = col_double(),
  smok = col_double(),
  air_polution = col_double(),
  doctor = col_double(),
  nurse_midwivies = col_double()
)



In [5]:
head(country_data)

country,age_1564,age_64up,age_0014,smok,air_polution,doctor,nurse_midwivies
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Afghanistan,54.32490,2.584927,43.09018,NA,56.91081,0.2907,NA
Albania,68.58239,13.744736,17.67287,28.7,18.20060,NA,NA
Algeria,63.48882,6.362497,30.14868,15.6,38.88401,NA,NA
American Samoa,NA,NA,NA,NA,12.47382,NA,NA
Andorra,NA,NA,NA,33.5,10.30762,3.3333,4.0128
Angola,50.97470,2.216374,46.80892,NA,32.38850,NA,NA


## 3. Explore the dataset

In [6]:
dim(covid19_confirmed)

[1] 264  99

In [7]:
colnames(covid19_confirmed)[length(covid19_confirmed)]

[1] "4/25/20"

In [8]:
dim(covid19_death)

[1] 264  99

In [9]:
colnames(covid19_death)[length(covid19_death)]

[1] "4/25/20"

In [10]:
test_that("Column names of covid19_confirmed and covid19_death should be the same.",
          {expect_true(all(colnames(covid19_confirmed) == colnames(covid19_death)))
          })

In [11]:
test_that("The country column of covid19_confirmed and covid19_death should be the same.",
          {expect_true(all(covid19_confirmed[2] == covid19_death[2]))
          })

In [12]:
test_that("The last column of covid19_confirmed should not have missing values.",
          {expect_true(all(!is.na(covid19_confirmed[length(covid19_confirmed)])))
          })

In [13]:
test_that("The last column of covid19_death should not have missing values.",
          {expect_true(all(!is.na(covid19_death[length(covid19_death)])))
          })

In [14]:
dim(country_data)

[1] 264   8

## 4. Initial thoughts


## 5. Wrangling

In [15]:
confirmed <- covid19_confirmed[c(2, length(covid19_confirmed))]
colnames(confirmed) = c("country", "n")

confirmed <- confirmed %>%
    group_by(country) %>%
    summarize(confirmed = sum(n))

In [16]:
death <- covid19_death[c(2, length(covid19_death))]
colnames(death) = c("country", "n")

death <- death %>%
    group_by(country) %>%
    summarize(death = sum(n))

In [17]:
covid19 <- merge(confirmed, death, by = "country") %>%
    mutate(rate = death / confirmed)
head(covid19)

country,confirmed,death,rate
<chr>,<dbl>,<dbl>,<dbl>
Afghanistan,1463,47,0.03212577
Albania,712,27,0.03792135
Algeria,3256,419,0.12868550
Andorra,738,40,0.05420054
Angola,25,2,0.08000000
Antigua and Barbuda,24,3,0.12500000


In [18]:
dim(covid19)

[1] 185   4

In [19]:
covid19 %>%
    filter(rate >= 1)

country,confirmed,death,rate
<chr>,<dbl>,<dbl>,<dbl>


In [20]:
covid19 %>%
    filter(!country %in% country_data$country)

country,confirmed,death,rate
<chr>,<dbl>,<dbl>,<dbl>
Bahamas,78,11,0.141025641
Brunei,138,1,0.007246377
Burma,146,5,0.034246575
Congo (Brazzaville),200,6,0.030000000
Congo (Kinshasa),416,28,0.067307692
Czechia,7352,218,0.029651795
Diamond Princess,712,13,0.018258427
Egypt,4319,307,0.071081269
Gambia,10,1,0.100000000


In [21]:
country_data <- country_data %>%
    mutate(country = case_when(country == 'Bahamas, The' ~ 'Bahamas',
                               country == 'Brunei Darussalam' ~ 'Brunei',
                               country == 'Egypt, Arab Rep.' ~ 'Egypt',
                               country == 'Gambia, The' ~ 'Gambia',
                               country == 'Iran, Islamic Rep.' ~ 'Iran',
                               country == 'Korea, Dem. People’s Rep.' ~ 'Korea, South',
                               country == 'Czech Republic' ~ 'Czechia',
                               country == 'Lao PDR' ~ 'Laos',
                               country == 'Russian Federation' ~ 'Russia',
                               country == 'St. Lucia' ~ 'Saint Lucia',
                               country == 'St. Vincent and the Grenadines' ~ 'Saint Vincent and the Grenadines',
                               country == 'Slovak Republic' ~ 'Slovakia',
                               country == 'Syrian Arab Republic' ~ 'Syria',
                               country == 'Venezuela, RB' ~ 'Venezuela',
                               country == 'Sub-Saharan Africa' ~ 'Western Sahara',
                               country == 'Yemen, Rep.' ~ 'Yemen',
                               TRUE ~ country))

In [22]:
covid19 <- covid19 %>%
    mutate(country = case_when(country == 'Burma' ~ 'Myanmar',
                               country == 'Congo (Brazzaville)' ~ 'Congo, Rep.',
                               country == 'Congo (Kinshasa)' ~ 'Congo, Dem. Rep.',
                               country == 'Kyrgyzstan' ~ 'Kyrgyz Republic',
                               country == 'Kyrgyzstan' ~ 'Kyrgyz Republic',
                               country == 'US' ~ 'United States',
                               TRUE ~ country))

In [23]:
covid19 %>%
    filter(!country %in% country_data$country)

country,confirmed,death,rate
<chr>,<dbl>,<dbl>,<dbl>
Diamond Princess,712,13,0.01825843
Holy See,9,0,0.00000000
MS Zaandam,9,2,0.22222222
Saint Kitts and Nevis,15,0,0.00000000
Taiwan*,429,6,0.01398601


In [24]:
data <- merge(country_data, covid19, by = "country")

In [25]:
head(data)

country,age_1564,age_64up,age_0014,smok,air_polution,doctor,nurse_midwivies,confirmed,death,rate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Afghanistan,54.32490,2.584927,43.09018,NA,56.91081,0.2907,NA,1463,47,0.03212577
Albania,68.58239,13.744736,17.67287,28.7,18.20060,NA,NA,712,27,0.03792135
Algeria,63.48882,6.362497,30.14868,15.6,38.88401,NA,NA,3256,419,0.12868550
Andorra,NA,NA,NA,33.5,10.30762,3.3333,4.0128,738,40,0.05420054
Angola,50.97470,2.216374,46.80892,NA,32.38850,NA,NA,25,2,0.08000000
Antigua and Barbuda,69.11908,8.799826,22.08109,NA,18.62234,NA,NA,24,3,0.12500000


In [33]:
drop_na(data)

,country,age_1564,age_64up,age_0014,smok,air_polution,doctor,nurse_midwivies,confirmed,death,rate
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9,Australia,65.15291,15.656475,19.19062,14.7,8.550324,3.5213,12.4667,6694,80,0.011951001
10,Austria,66.70049,19.001566,14.29795,29.6,12.477967,5.0701,8.1758,15148,536,0.035384209
13,Bahrain,78.31936,2.426334,19.25431,26.4,70.816208,0.9257,2.4944,2588,8,0.003091190
14,Bangladesh,67.13559,5.158391,27.70602,23.0,60.845785,0.4709,0.2665,4998,140,0.028011204
17,Belgium,64.15583,18.788744,17.05542,28.2,12.886518,3.0138,10.8193,45325,6917,0.152608935
25,Brunei,72.10039,4.873148,23.02646,16.9,5.903065,1.7701,6.6012,138,1,0.007246377
29,Cabo Verde,66.61291,4.609327,28.77777,9.1,34.778700,0.7694,1.2272,90,1,0.011111111
32,Canada,66.89774,17.232007,15.87025,14.3,6.428383,2.5388,9.8398,45491,2547,0.055989097
36,China,71.20211,10.920884,17.87700,25.6,52.664596,1.7855,2.3073,83909,4636,0.055250331


## 6. Research questions



## 7. Data Analysis & Visualizations

In [36]:
fit <- lm(rate ~ ., drop_na(data)[c(-1, -2, -4, -9, -10)])
fit %>%
    broom::tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.0126676901,0.0188622789,0.6715885,0.5044233299
age_64up,0.0050672295,0.0012844502,3.9450571,0.0002116083
smok,-0.0003567535,0.0005786309,-0.6165476,0.5398655547
air_polution,0.0002083572,0.0003442810,0.6051951,0.5473316181
doctor,-0.0104240619,0.0062209343,-1.6756425,0.0990125499
nurse_midwivies,-0.0004123217,0.0016039963,-0.2570590,0.7980127033


In [38]:
anova(fit)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
age_64up,1,1.867756e-02,1.867756e-02,13.76629064,0.0004548139
smok,1,8.338623e-04,8.338623e-04,0.61459808,0.4361458261
air_polution,1,1.119795e-03,1.119795e-03,0.82534441,0.3672569408
doctor,1,5.703673e-03,5.703673e-03,4.20389168,0.0447077574
nurse_midwivies,1,8.965383e-05,8.965383e-05,0.06607934,0.7980127033
Residuals,60,8.140562e-02,1.356760e-03,NA,NA


In [40]:
fit_base <- lm(rate ~ age_64up, drop_na(data))
fit_base %>%
    broom::tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.012071280,0.0091360741,1.321277,0.1911135184
age_64up,0.002543938,0.0006947418,3.661702,0.0005096342


In [41]:
fit_1 <- lm(rate ~ age_64up + doctor, drop_na(data))
fit_1 %>%
    broom::tidy()

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.015853042,0.009008437,1.759799,0.0832963278
age_64up,0.004597708,0.001127520,4.077717,0.0001299839
doctor,-0.011971537,0.005272208,-2.270687,0.0265942204


In [43]:
anova(fit_1, fit_base)

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,0.08240819,NA,NA,NA,NA
64,0.08915260,-1,-0.006744418,5.156021,0.02659422


## 8. Summary and conclusions
